In [6]:
%%time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set device: Use GPU if available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 1
batch_size = 640
learning_rate = 0.01

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the MNIST training dataset.
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

# Create a DataLoader with asynchronous loading:
# - num_workers > 0 spawns subprocesses to load data in parallel.
# - pin_memory=True enables faster host-to-GPU transfer.
# - persistent_workers=True (if supported) keeps workers alive across epochs.
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

# Define a simple neural network model.
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        return self.fc(x)

model = SimpleNN().to(device)

# Loss function and optimizer.
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop.
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        # Transfer batch to GPU asynchronously.
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        # Forward pass.
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
            
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] finished, Average Loss: {avg_loss:.4f}")


Epoch [1/1] finished, Average Loss: 1.6244
CPU times: total: 500 ms
Wall time: 4.12 s


In [8]:
import time
t=time.time()
next(iter(train_loader))
print(time.time()-t)
t=time.time()
next(iter(train_loader))
print(time.time()-t)
t=time.time()
next(iter(train_loader))
print(time.time()-t)
time.sleep(1)
t=time.time()
next(iter(train_loader))
print(time.time()-t)

0.058508872985839844
0.15851116180419922
0.1620168685913086
0.05400252342224121


In [11]:
from dataclasses import dataclass, field, fields
@dataclass(slots=True)
class test:
    a: int = field(init=False, default=5)

In [12]:
t=test()
print(t)
t.a=5
print(t)
t.b=6

test(a=5)
test(a=5)


AttributeError: 'test' object has no attribute 'b'